In [736]:
import numpy as np
import pandas as pd
from dbfread import DBF

## Data Import

In [737]:
def read_dbf(filename: str) -> pd.DataFrame:
  return pd.DataFrame(DBF(filename))

In [738]:
# MEMBERS
df_members = read_dbf('./data/members.DBF')

# EXPEDITIONS
df_exped = read_dbf('./data/exped.DBF')

# PEAKS
df_peaks = read_dbf('./data/peaks.DBF')

## Data Cleaning

In [739]:
def standardize_colnames(df: pd.DataFrame) -> pd.DataFrame:
  df.columns = df.columns.str.lower()

### Members

In [740]:
df_members.head()

,EXPID,MEMBID,PEAKID,MYEAR,MSEASON,FNAME,LNAME,SEX,AGE,BIRTHDATE,...,MEMBERMEMO,NECROLOGY,MSMTBID,MSMTTERM,HCN,MCHKSUM,MSMTNOTE1,MSMTNOTE2,MSMTNOTE3,DEATHRTE
0,AMAD78301,01,AMAD,1978,3,Jean Robert,Clemenson,M,0,None,...,None,None,1,4,0,2426937,NaN,NaN,NaN,NaN
1,AMAD78301,02,AMAD,1978,3,Bernard,Dufour,M,0,None,...,None,None,1,4,0,2426501,NaN,NaN,NaN,NaN
2,AMAD78301,03,AMAD,1978,3,Philippe,Gerard,M,0,None,...,None,None,1,4,0,2431569,NaN,NaN,NaN,NaN
3,AMAD78301,04,AMAD,1978,3,Eric,Lasserre,M,0,None,...,None,None,1,4,0,2426809,NaN,NaN,NaN,NaN
4,AMAD78301,05,AMAD,1978,3,Guy,Peters,M,0,None,...,None,None,1,4,0,2429215,NaN,NaN,NaN,NaN


In [741]:
standardize_colnames(df_members)

In [742]:
columns = list(df_members.columns)

In [743]:
col_list = [
  # KEY COLUMNS
  'expid',
  'membid',
  'peakid',
  # EXPEDITION TIME
  'myear',
  'mseason',
  # MEMBER PERSONAL DETAILS
  # 'fname',
  # 'lname',
  'sex',
  # 'age',
  # 'birthdate',
  # 'yob',
  'calcage',
  # 'citizen',
  'status',
  # 'residence',
  # 'occupation',
  # MEMBER ROLE
  'leader',
  'deputy',
  'bconly',
  'nottobc',
  'support',
  'disabled',
  'hired',
  'sherpa',
  'tibetan',
  # EXPEDITION OUTCOME
  'msuccess',
  # 'mclaimed',
  # 'mdisputed',
  # EXPEDITION TYPE
  'msolo',
  'mtraverse',
  'mski',
  'mparapente',
  'mspeed',
  # SUMMIT BID DETAILS
  'mhighpt',
  'mperhighpt',
  # 'msmtdate1',
  # 'msmtdate2',
  # 'msmtdate3',
  # 'msmttime1',
  # 'msmttime2',
  # 'msmttime3',
  # EXPEDITION ROUTE/ASCENT
  'mroute1',
  'mroute2',
  'mroute3',
  'mascent1',
  'mascent2',
  'mascent3',
  # OXYGEN USE
  'mo2used',
  'mo2none',
  'mo2climb',
  'mo2descent',
  'mo2sleep',
  'mo2medical',
  'mo2note',
  # CLIMBER DEATH
  'death',
  # 'deathdate',
  # 'deathtime',
  'deathtype',
  # 'deathhgtm',
  'deathclass',
  'ams',
  'weather',
  # CLIMBER INJURY
  'injury',
  # 'injurydate',
  # 'injurytime',
  'injurytype',
  # 'injuryhgtm',
  # SUMMIT BID
  'msmtbid',
  'msmtterm'
]

In [744]:
df_members = df_members[col_list]
df_members.rename(columns={'calcage': 'mage'}, inplace=True)

In [745]:
df_members.shape

(87156, 47)

In [746]:
df_members = df_members.loc[
  (df_members.bconly == False) &
  (df_members.nottobc == False) &
  (df_members.support == False) &
  (df_members.hired == False) &
  (df_members.sherpa == False) &
  (df_members.tibetan == False)
]

df_members.drop(['bconly', 'nottobc', 'support', 'disabled', 'hired', 'sherpa', 'tibetan'], axis=1, inplace=True)

In [747]:
df_members = df_members.loc[
  (df_members.mtraverse == False) &
  (df_members.mski == False) &
  (df_members.mparapente == False) &
  (df_members.mspeed == False),
  :
]

df_members.drop(['mtraverse', 'mski', 'mparapente', 'mspeed'], axis=1, inplace=True)

In [748]:
df_members.status = df_members.status.str.lower()

df_members = df_members.loc[
  (df_members.status.str.contains('climb')) |
  (df_members.status.str.contains('lead')),
  :
]

df_members = df_members.loc[
  (~df_members.status.str.contains('non-climber')) &
  (~df_members.status.str.contains('ski')) &
  (~df_members.status.str.contains('paraglider')) &
  (~df_members.status.str.contains('camera')) &
  (~df_members.status.str.contains('photo')) &
  (~df_members.status.str.contains('film')) &
  (~df_members.status.str.contains('reporter')) &
  (~df_members.status.str.contains('journalist')) &
  (~df_members.status.str.contains('coach')) &
  (~df_members.status.str.contains('advisor')) &
  (~df_members.status.str.contains('instructor')) &
  (~df_members.status.str.contains('support')) &
  (~df_members.status.str.contains('guide')) &
  (~df_members.status.str.contains('torch')) &
  (~df_members.status.str.contains('only'))
]

df_members.drop(['status'], axis=1, inplace=True)

In [749]:
df_members.head()

,expid,membid,peakid,myear,mseason,sex,mage,leader,deputy,msuccess,...,mo2note,death,deathtype,deathclass,ams,weather,injury,injurytype,msmtbid,msmtterm
56,AMAD79303,03,AMAD,1979,3,M,35,False,False,False,...,,False,0,0,False,False,False,0,1,14
192,AMAD85101,02,AMAD,1985,1,M,20,False,False,True,...,,False,0,0,False,False,False,0,5,1
209,AMAD85301,07,AMAD,1985,3,M,28,False,False,True,...,,False,0,0,False,False,False,0,5,1
213,AMAD85303,02,AMAD,1985,3,M,41,False,False,True,...,,False,0,0,False,False,False,0,5,1
458,AMAD90301,06,AMAD,1990,3,F,39,False,False,False,...,,False,0,0,False,False,False,0,1,4


In [750]:
df_members.shape

(25279, 35)

In [751]:
df_members.columns

Index(['expid', 'membid', 'peakid', 'myear', 'mseason', 'sex', 'mage',
       'leader', 'deputy', 'msuccess', 'msolo', 'mhighpt', 'mperhighpt',
       'mroute1', 'mroute2', 'mroute3', 'mascent1', 'mascent2', 'mascent3',
       'mo2used', 'mo2none', 'mo2climb', 'mo2descent', 'mo2sleep',
       'mo2medical', 'mo2note', 'death', 'deathtype', 'deathclass', 'ams',
       'weather', 'injury', 'injurytype', 'msmtbid', 'msmtterm'],
      dtype='object')

In [752]:
df_members.mascent1.unique()

array([  0,  36,  42,  40, 110, 218, 233,  38,  43,  45, 145, 287,  12,
         6, 171, 189,   8,   7,   5,  41,   2,  10, 241,  23,  17,  76,
        78,  13, 353, 343, 327,   4, 387, 398, 434, 415, 397,   1,   3,
        65,  99,  84, 128, 131,  32,  49,  79,  77, 132, 133, 135, 138,
       140, 134, 137, 136, 141, 139,  71, 121, 117, 125, 118, 124, 122,
       114, 112, 116, 119, 126, 127, 123, 113, 115, 111,  94,  89,  92,
        96,  91, 107,  98,  93, 104,  87, 108, 105, 100,  88, 106, 109,
       102, 103,  97,  90, 101,  95,  29,  31,  30,  81,  20,  21,  22,
        86,  82,  80,  85,  83,  25,  24,  14,  33, 271,  50, 147, 148,
       143, 144, 149, 150,  73,  74,  75, 129, 156,   9,  28,  26,  27,
        34,  35,  15,  51,  54,  53,  52, 151, 152, 120,  37,  16, 157,
       158,  56,  55,  57,  39,  44,  46,  48,  47, 279,  11])

#### Data Checks

In [753]:
# a single entry per expedition/member combination
df_members.groupby(['expid', 'membid']).size().max() == 1

True

In [754]:
# member success does not match summit bid entry
df_members.loc[df_members.msuccess != (df_members.msmtbid == 5)]

,expid,membid,peakid,myear,mseason,sex,mage,leader,deputy,msuccess,...,mo2note,death,deathtype,deathclass,ams,weather,injury,injurytype,msmtbid,msmtterm
45648,MANA08105,01,MANA,2008,1,M,35,True,False,False,...,,False,0,0,False,False,False,0,5,2
45660,MANA08108,02,MANA,2008,1,F,63,False,False,False,...,From C3 to smt to C3,False,0,0,False,False,False,0,5,2
45661,MANA08108,01,MANA,2008,1,M,46,True,False,False,...,From C4 to smt to C4,False,0,0,False,False,False,0,5,2
45662,MANA08108,03,MANA,2008,1,F,59,False,False,False,...,From C3 to smt to C3,False,0,0,False,False,False,0,5,2
45663,MANA08108,04,MANA,2008,1,F,47,False,False,False,...,From C3 to smt to C3,False,0,0,False,False,False,0,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85141,LOBE23101,02,LOBE,2023,1,F,40,False,False,False,...,,False,0,0,False,False,False,0,5,2
87119,ROLK23301,01,ROLK,2023,3,M,60,False,False,False,...,,False,0,0,False,False,False,0,5,2
87120,ROLK23301,02,ROLK,2023,3,M,41,False,False,False,...,,False,0,0,False,False,False,0,5,2
87140,TENR23301,02,TENR,2023,3,M,40,False,False,False,...,,False,0,0,False,False,False,0,5,2


### Peaks

In [755]:
df_peaks.shape

(479, 25)

In [756]:
standardize_colnames(df_peaks)

In [757]:
columns = list(df_peaks.columns)
columns

['peakid',
 'pkname',
 'pkname2',
 'location',
 'heightm',
 'heightf',
 'himal',
 'region',
 'open',
 'unlisted',
 'trekking',
 'trekyear',
 'restrict',
 'phost',
 'pstatus',
 'peakmemo',
 'pyear',
 'pseason',
 'pexpid',
 'psmtdate',
 'pcountry',
 'psummiters',
 'psmtnote',
 'refermemo',
 'photomemo']

In [758]:
col_list = [
  # ID COLUMN
  'peakid',
  # PEAK INFO
  'location',
  'heightm',
  'himal',
  'region',
  # PEAK STATUS
  'open',
  'unlisted',
  'trekking',
  'phost',
  'pstatus'
]

In [759]:
df_peaks = df_peaks[col_list]

In [760]:
df_peaks = df_peaks.loc[df_peaks.trekking == False]
df_peaks.drop('trekking', axis=1, inplace=True)

In [761]:
df_peaks.shape

(448, 9)

In [762]:
df_peaks.head()

,peakid,location,heightm,himal,region,open,unlisted,phost,pstatus
0,AMAD,Khumbu Himal,6814,12,2,True,False,1,2
1,AMPG,Khumbu Himal (N of Ama Dablam),5630,12,2,True,False,1,2
2,ANN1,Annapurna Himal,8091,1,5,True,False,1,2
3,ANN2,Annapurna Himal,7937,1,5,True,False,1,2
4,ANN3,Annapurna Himal,7555,1,5,True,False,1,2


### Expeditions

In [763]:
standardize_colnames(df_exped)

In [764]:
df_exped.head()

,expid,peakid,year,season,host,route1,route2,route3,route4,nation,...,accidents,achievment,agency,comrte,stdrte,primrte,primmem,primref,primid,chksum
0,ANN260101,ANN2,1960,1,1,NW Ridge-W Ridge,,,,UK,...,,,,None,None,False,False,None,,2442047
1,ANN269301,ANN2,1969,3,1,NW Ridge-W Ridge,,,,Yugoslavia,...,Draslar frostbitten hands and feet,,,None,None,False,False,None,,2445501
2,ANN273101,ANN2,1973,1,1,W Ridge-N Face,,,,Japan,...,,,,None,None,False,False,None,,2446797
3,ANN278301,ANN2,1978,3,1,N Face-W Ridge,,,,UK,...,,,,None,None,False,False,None,,2448822
4,ANN279301,ANN2,1979,3,1,N Face-W Ridge,NW Ridge of A-IV,,,UK,...,,,,None,None,False,False,None,,2449204


In [765]:
columns = list(df_exped.columns)

In [766]:
col_list = [
  # ID COLUMNS
  'expid',
  'peakid',
  # EXPEDICTION TIME
  'year',
  'season',
  'host',
  # EXPEDITION ROUTE
  'route1',
  # 'route2',
  # 'route3',
  # 'route4',
  # EXPEDITION TEAM INFO
  # 'nation', # principle nationality
  # 'leaders',
  'sponsor',
  # EXPEDITION OUTCOME
  'success1', # success on route 1
  'success2',
  'success3',
  'success4',
  # 'ascent1', # ascent numbers for route 1
  # 'ascent2',
  # 'ascent3',
  # 'ascent4',
  # 'claimed',
  # 'disputed',
  'countries',
  # ASCENT DETAILS
  'approach',
  # 'bcdate',
  # 'smtdate',
  # 'smttime',
  # 'smtdays',
  # 'totdays',
  # 'termdate',
  'termreason',
  # 'termnote',
  # 'highpoint',
  # EXPEDITION TYPE
  'traverse',
  'ski',
  'parapente',
  # EQUIPMENT
  'camps',
  'rope',
  # TEAM SIZE
  'totmembers',
  'smtmembers',
  'mdeaths',
  'tothired',
  'smthired',
  'hdeaths',
  'nohired',
  # OXYGEN USE
  'o2used',
  'o2none',
  'o2climb',
  'o2descent',
  'o2sleep',
  'o2medical',
  'o2taken',
  'o2unkwn',
  # OTHER DETAILS
  # 'othersmts',
  # 'campsites',
  # 'routememo',
  # 'accidents',
  # 'achievment',
  # MISC
  'agency',
  'comrte',
  'stdrte',
  'primrte',
  'primmem',
  'primref',
  # 'primid',
  # 'chksum'
]

In [767]:
df_exped.shape[1] == len(columns)

True

In [768]:
df_exped = df_exped[col_list]

In [769]:
df_exped = df_exped.loc[
  (df_exped.traverse == False) &
  (df_exped.parapente == False) &
  (df_exped.ski == False)
]
df_exped.drop(['traverse', 'parapente', 'ski'], axis=1, inplace=True)

In [770]:
df_exped.shape

(10997, 37)

In [771]:
df_exped.head()

,expid,peakid,year,season,host,route1,sponsor,success1,success2,success3,...,o2sleep,o2medical,o2taken,o2unkwn,agency,comrte,stdrte,primrte,primmem,primref
0,ANN260101,ANN2,1960,1,1,NW Ridge-W Ridge,,True,False,False,...,True,False,False,False,,None,None,False,False,None
1,ANN269301,ANN2,1969,3,1,NW Ridge-W Ridge,Mountaineering Club of Slovenia,True,False,False,...,False,False,False,False,,None,None,False,False,None
2,ANN273101,ANN2,1973,1,1,W Ridge-N Face,Sangaku Doshikai Annapurna II Expedition 1973,True,False,False,...,False,False,False,False,,None,None,False,False,None
3,ANN278301,ANN2,1978,3,1,N Face-W Ridge,British Annapurna II Expedition,False,False,False,...,False,False,False,False,,None,None,False,False,None
4,ANN279301,ANN2,1979,3,1,N Face-W Ridge,,False,False,False,...,False,False,False,False,,None,None,False,False,None


In [772]:
df_exped.route1.isna().sum()

0

In [773]:
df_exped['success'] = df_exped.apply(lambda row: row.success1 or row.success2 or row.success3 or row.success4, axis=1)
df_exped.drop(['success1', 'success2', 'success3', 'success4'], axis=1, inplace=True)

In [774]:
df_exped.columns

Index(['expid', 'peakid', 'year', 'season', 'host', 'route1', 'sponsor',
       'countries', 'approach', 'termreason', 'camps', 'rope', 'totmembers',
       'smtmembers', 'mdeaths', 'tothired', 'smthired', 'hdeaths', 'nohired',
       'o2used', 'o2none', 'o2climb', 'o2descent', 'o2sleep', 'o2medical',
       'o2taken', 'o2unkwn', 'agency', 'comrte', 'stdrte', 'primrte',
       'primmem', 'primref', 'success'],
      dtype='object')

### Data Merge

In [775]:
df_members.columns

Index(['expid', 'membid', 'peakid', 'myear', 'mseason', 'sex', 'mage',
       'leader', 'deputy', 'msuccess', 'msolo', 'mhighpt', 'mperhighpt',
       'mroute1', 'mroute2', 'mroute3', 'mascent1', 'mascent2', 'mascent3',
       'mo2used', 'mo2none', 'mo2climb', 'mo2descent', 'mo2sleep',
       'mo2medical', 'mo2note', 'death', 'deathtype', 'deathclass', 'ams',
       'weather', 'injury', 'injurytype', 'msmtbid', 'msmtterm'],
      dtype='object')

In [776]:
df_peaks.columns

Index(['peakid', 'location', 'heightm', 'himal', 'region', 'open', 'unlisted',
       'phost', 'pstatus'],
      dtype='object')

In [777]:
df_exped.groupby(['expid', 'peakid']).size().reset_index().sort_values(0, ascending=False)

,expid,peakid,0
7451,KANG10101,KANG,2
6105,EVER22101,EVER,2
6053,EVER21101,EVER,2
1,ACHN15302,ACHN,1
2,ACHN18301,ACHN,1
...,...,...,...
10957,URMA04301,URMA,1
10958,URMA07301,URMA,1
10959,URMA10101,URMA,1
10960,URMA10401,URMA,1


In [778]:
df = df_members.merge(df_peaks, how='inner', on=['peakid'])
df = df.merge(df_exped, how='inner', left_on=['expid', 'peakid', 'myear', 'mseason'], right_on=['expid', 'peakid', 'year', 'season'])

In [779]:
df.shape

(24699, 75)

In [781]:
df.msuccess.value_counts()

msuccess
False    13735
True     10964
Name: count, dtype: int64